In [1]:
import pandas as pd

# Load the data
file_path = "teleconnect.csv"
data = pd.read_csv(file_path)

# Quick look at the structure and contents of the file
data_info = data.info()
head = data.head()
sample = data.sample(5)

(data_info, head, sample)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


(None,
    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
 0  7590-VHVEG  Female              0     Yes         No       1           No   
 1  5575-GNVDE    Male              0      No         No      34          Yes   
 2  3668-QPYBK    Male              0      No         No       2          Yes   
 3  7795-CFOCW    Male              0      No         No      45           No   
 4  9237-HQITU  Female              0      No         No       2          Yes   
 
       MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
 0  No phone service             DSL             No  ...               No   
 1                No             DSL            Yes  ...              Yes   
 2                No             DSL            Yes  ...               No   
 3  No phone service             DSL            Yes  ...              Yes   
 4                No     Fiber optic             No  ...               No   
 
   TechSupport StreamingTV StreamingMovie

Let’s get this data cleaned up and prepped. I need to:

Handle missing or incorrect data types (like "TotalCharges" being stored as an object).
Encode categorical variables.
Normalize numerical data for better ANN performance.
.....
The data’s all cleaned up and ready to roll:

Categorical variables are one-hot encoded.
Numerical columns are normalized.
“TotalCharges” is converted and fixed for any missing values.
We’ve got 31 columns, and the target ("Churn_Yes") is separated from the features.

Let’s move to building and optimizing the artificial neural network

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

file_path = "teleconnect.csv"
data = pd.read_csv(file_path)

# Convert TotalCharges to numeric, handling empty strings
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# Check for missing values and fill them if needed
data.fillna({'TotalCharges': data['TotalCharges'].median()}, inplace=True)

# Drop customerID as it's not useful for modeling
data.drop(columns=['customerID'], inplace=True)

# Encode categorical variables
cat_cols = data.select_dtypes(include='object').columns
data = pd.get_dummies(data, columns=cat_cols, drop_first=True)

# Normalize numerical columns
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
data[num_cols] = scaler.fit_transform(data[num_cols])

# Splitting features and target
X = data.drop(columns=['Churn_Yes'])
y = data['Churn_Yes']

# Confirming data is ready
data.info(), data.head()

# Splitting features and target
X = data.drop(columns=['Churn_Yes'])
y = data['Churn_Yes']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Build baseline ANN model
baseline_model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   float64
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   gender_Male                            7043 non-null   bool   
 5   Partner_Yes                            7043 non-null   bool   
 6   Dependents_Yes                         7043 non-null   bool   
 7   PhoneService_Yes                       7043 non-null   bool   
 8   MultipleLines_No phone service         7043 non-null   bool   
 9   MultipleLines_Yes                      7043 non-null   bool   
 10  InternetService_Fiber optic            7043 non-null   bool   
 11  Inte

/Users/rushita/Desktop/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
Let us Optimize with RMSprop and see if we can push that accuracy form 79.56%!

In [7]:
# Compile the model with RMSprop optimizer
baseline_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = baseline_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = baseline_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.8190 - loss: 0.3937 - val_accuracy: 0.7928 - val_loss: 0.4246
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - accuracy: 0.8195 - loss: 0.3896 - val_accuracy: 0.7942 - val_loss: 0.4200
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.8074 - loss: 0.4098 - val_accuracy: 0.7906 - val_loss: 0.4248
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.8154 - loss: 0.3966 - val_accuracy: 0.7991 - val_loss: 0.4206
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.8271 - loss: 0.3822 - val_accuracy: 0.7913 - val_loss: 0.4201
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.8191 - loss: 0.3992 - val_accuracy: 0.7942 - val_loss: 0.4214
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step - accuracy: 0.8298 - loss: 0.3805 - val_accuracy: 0.7878 - val_loss: 0.4227
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - accuracy: 0.8174 - loss: 0.3986 - 

In [ ]:
Switched the optimizer to SGD — to see how it performs!

In [8]:
# Compile the model with SGD optimizer
baseline_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = baseline_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = baseline_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - accuracy: 0.8271 - loss: 0.3760 - val_accuracy: 0.7885 - val_loss: 0.4240
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.8229 - loss: 0.3757 - val_accuracy: 0.7913 - val_loss: 0.4244
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.8267 - loss: 0.3784 - val_accuracy: 0.7906 - val_loss: 0.4257
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - accuracy: 0.8331 - loss: 0.3694 - val_accuracy: 0.7899 - val_loss: 0.4250
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.8254 - loss: 0.3747 - val_accuracy: 0.7899 - val_loss: 0.4253
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - accuracy: 0.8277 - loss: 0.3723 - val_accuracy: 0.7899 - val_loss: 0.4258
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - accuracy: 0.8260 - loss: 0.3790 - val_accuracy: 0.7913 - val_loss: 0.4250
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - accuracy: 0.8293 - loss: 0.3705 - 

In [ ]:
Switched the optimizer to Adam to see how it performs!

In [13]:
# Compile the model
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = baseline_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = baseline_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - accuracy: 0.8467 - loss: 0.3342 - val_accuracy: 0.7764 - val_loss: 0.4759
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 0.8403 - loss: 0.3364 - val_accuracy: 0.7757 - val_loss: 0.4720
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.8488 - loss: 0.3263 - val_accuracy: 0.7764 - val_loss: 0.4728
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.8471 - loss: 0.3341 - val_accuracy: 0.7771 - val_loss: 0.4726
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - accuracy: 0.8508 - loss: 0.3317 - val_accuracy: 0.7736 - val_loss: 0.4793
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - accuracy: 0.8444 - loss: 0.3297 - val_accuracy: 0.7715 - val_loss: 0.4871
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - accuracy: 0.8426 - loss: 0.3366 - val_accuracy: 0.7700 - val_loss: 0.4792
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 0.8451 - loss: 0.3266 - 